In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer
import xgboost as xgb
from matplotlib import pyplot as plt

import sys
sys.path.append('..')
from utils import preprocess, missing, evaluate

### Globals and load data

In [2]:
target = 'SI.POV.DDAY'
predict_year=2010
#percent of input Indicators to use (set to 100 for full set of input features)
percent = 50

#Load the data from disk
input_dir = '.\\..\\data\\'
data_input = "cleaned_data.pkl"
data = pd.read_pickle(input_dir + data_input)

#Possible subset of data choosen to reduce calulation time
#For percetages less than 100% we try to choose a subset that represents the spread of variables

if percent == 100:
    pass
else: 
    num_indicators_original = data.shape[1]
    step = int(100/percent)
    data_new = data.iloc[:,::step].copy()
    #Add the target column if not already included
    if target not in data_new.columns:
        data_new[target] = data[target]
    data = data_new
    
print(data.shape[1], "indicators included")

411 indicators included


### Window data and preprocess

In [3]:
%time data_regressors, data_targets = \
        preprocess.window_data(data, lag=3,num_windows=10, step=1, predict_year=2010, \
                         target=target, impute_type='interpolation')

#Break up into training and testing data.

idx = pd.IndexSlice
data_train_regressors = data_regressors.loc[idx[:,2:10],:]
data_train_targets = data_targets.loc[idx[:,2:10],:]
data_test_regressors = data_regressors.loc[idx[:,1],:]
data_test_targets= data_targets.loc[idx[:,1],:]

#For Training, only consider windows that don't have a missing target as they offer nothing to training
#Therefore, remove those observations from both the training regressors and targets datasets.
data_train_regressors_subset = data_train_regressors[~np.isnan(list(data_train_targets.values.flatten()))]
data_train_targets_subset = data_train_targets[~np.isnan(list(data_train_targets.values.flatten()))]

#For testing, also remove windows with no target variable as it is impossible to measure preformance.
data_test_regressors_subset = data_test_regressors[~np.isnan(list(data_test_targets.values.flatten()))]
data_test_targets_subset = data_test_targets[~np.isnan(list(data_test_targets.values.flatten()))]

Wall time: 3min 26s


In [4]:
X_train = data_train_regressors_subset.values
y_train = data_train_targets_subset.values.ravel()
X_test = data_test_regressors_subset.values
y_test = data_test_targets_subset

### XGBoost Model

#### Out-of-the-box using the Scikit-learn interface

In [47]:
XGB = xgb.XGBRegressor(random_state=42 ,objective='reg:squarederror', subsample=0.9)
XGB.fit( X_train,y_train)
#Make predictions
predictions = XGB.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of XGBoost out-of-the-box is:", np.sqrt(mse))

RMSE of XGBoost out-of-the-box is: 5.154362890679558


#### Tuning of the Algorithm

In [54]:
cv_folds = 5

scorer = make_scorer(mean_squared_error ,greater_is_better=False)

Step 1. Tune the number of estimators

In [55]:
model = xgb.XGBRegressor(random_state=42,
                         objective='reg:squarederror',
                         max_depth=5, 
                         min_child_weight = 1, 
                         gamma = 0, 
                         subsample=0.9, 
                         colsample_bytree = 0.8, 
                         scale_pos_weight = 1)

param = model.get_xgb_params()
data_matrix = xgb.DMatrix(X_train, label=y_train)
cvresult = xgb.cv(param, data_matrix, num_boost_round=model.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=50)
#Set the optimised number of estimators
model.set_params(n_estimators=cvresult.shape[0])
print("Optimal number of estimators:", cvresult.shape[0])

Optimal number of estimators: 100


In [56]:
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 1) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 1) is: 5.189437244183729


Step 2. Tune max_depth and min_child_weight

In [57]:
params = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 2) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 2) is: 5.494248333090531


The result of our test shows that the performance of the model after tuning was actually worse than before. The model is generalising very poorly. This may be a reflection on some kind of bias introduced in creating our training and data subsets. It would be worth looking at how I decided to discard any countries early on that did not have target values for the target year, 2010. It may have made more sense to window the data and split into training and test subsets and then, after, discard any observations that did not have a target value. 

In [60]:
grid_model.best_params_

{'max_depth': 5, 'min_child_weight': 5}

Step 3. Tune Gamma

In [61]:
params = {
 'gamma':[i/10.0 for i in range(0,5)]
}



grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 3) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 3) is: 5.141798455470761


In [62]:
grid_model.best_params_

{'gamma': 0.0}

Step 4. Tune Regularization

In [63]:
params = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1]
}

grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5 ,return_train_score=True)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 4) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 4) is: 5.141798730456352


In [64]:
grid_model.best_params_

{'reg_alpha': 1e-05}

In [65]:
params = {
 'reg_lambda':[1e-5, 1e-2, 0.1, 1]
}

grid_model = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5 ,return_train_score=True)
grid_model.fit(X_train,y_train)

#Score the best model using the test data
model = grid_model.best_estimator_
model.fit( X_train,y_train)
#Make predictions
predictions = model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("RMSE of xgboost after tuning (step 5) is:", np.sqrt(mse))

RMSE of xgboost after tuning (step 5) is: 5.141798730456352


### XGBoost Model with no Imputation

In [66]:
%time data_regressors, data_targets = \
        preprocess.window_data(data, lag=3,num_windows=10, step=1, predict_year=2010, \
                         target=target)

#Break up into training and testing data.

idx = pd.IndexSlice
data_train_regressors = data_regressors.loc[idx[:,2:10],:]
data_train_targets = data_targets.loc[idx[:,2:10],:]
data_test_regressors = data_regressors.loc[idx[:,1],:]
data_test_targets= data_targets.loc[idx[:,1],:]

#For Training, only consider windows that don't have a missing target as they offer nothing to training
#Therefore, remove those observations from both the training regressors and targets datasets.
data_train_regressors_subset = data_train_regressors[~np.isnan(list(data_train_targets.values.flatten()))]
data_train_targets_subset = data_train_targets[~np.isnan(list(data_train_targets.values.flatten()))]

#For testing, also remove windows with no target variable as it is impossible to measure preformance.
data_test_regressors_subset = data_test_regressors[~np.isnan(list(data_test_targets.values.flatten()))]
data_test_targets_subset = data_test_targets[~np.isnan(list(data_test_targets.values.flatten()))]

X_train_miss = data_train_regressors_subset.values
y_train_miss = data_train_targets_subset.values.ravel()
X_test_miss = data_test_regressors_subset.values
y_test_miss = data_test_targets_subset


Wall time: 2.88 s


#### Out-of-the-box xgboost on data without imputation

Hand-tuning of the XGBoost model. I pick some important paramneters and play around until I get a good result. I'm sure there is more accuracy that can be obtained from this model by gridsearching but I think this is enough to illustrate that using a XGBoost (or perhaps any tree-based predictive algo) without any imputation done on the input data gives by far the best results of any of the models tried.

In [135]:
XGB = xgb.XGBRegressor(n_estimators=200,  objective='reg:squarederror',max_depth=7, subsample=0.87, reg_lambda=0.2)
XGB.fit( X_train_miss,y_train_miss)
#Make predictions
predictions = XGB.predict(X_test_miss) 

mse= mean_squared_error(y_test_miss, predictions)
print("RMSE of XGBoost out-of-the-box is:", np.sqrt(mse))

RMSE of XGBoost out-of-the-box is: 3.0772104636217508
